In [4]:
from IPython.display import display
import spotipy
import spotipy.util as util

client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []

offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break
    offset += 50
print("All tracks fetched")

artist_names = {}
artist_tracks = {}
all_artists = set()
for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    track_id = tracks[i]['id']
    artist_id = tracks[i]['artists'][0]['id']
    
    if artist_id not in artist_tracks:
        artist_tracks[artist_id] = [track_id]
    else:
        artist_tracks[artist_id].append(track_id)
    
    for artist in tracks[i]['artists']:
        all_artists.add(artist['id'])
        artist_names[artist['id']] = artist['name']
print("Artists fetched")

All tracks fetched
Artists fetched


In [5]:
import networkx as nx

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

artist_genres = {}
for chunk in chunks(50, list(artist_tracks.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']
print("Genres fetched")

try:
    G = nx.read_graphml('related_artists.graphml')
except FileNotFoundError:
    G = nx.Graph()
    
i = 0
not_found = 0
for artist_id in all_artists:
    if not G.has_node(artist_id):
        not_found += 1
        related_list = sp.artist_related_artists(artist_id)['artists']

        for related in related_list:
            artist_names[related['id']] = artist['name']
            G.add_edge(artist_id, related['id'])

            sub_related_list = sp.artist_related_artists(related['id'])['artists']
            for sub_related in sub_related_list:
                artist_names[sub_related['id']] = artist['name']
                G.add_edge(related_id, sub_related['id'])

    i += 1
    if i % 500 ==0:
        print("%d/%d artists processed, %d fetched" % (i, len(all_artists), not_found))
        
for artist in list(artist_tracks.keys()):
    if not G.has_node(artist):
        del artist_tracks[artist]
        
nx.write_graphml(G, 'related_artists.graphml')
print("All artists processed")

Genres fetched
500/2568 artists processed, 14 fetched
1000/2568 artists processed, 30 fetched
1500/2568 artists processed, 55 fetched
2000/2568 artists processed, 81 fetched
2500/2568 artists processed, 111 fetched
All artists processed


In [15]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

artists = list(artist_tracks.keys())
distance = np.zeros((len(artists), len(artists)))
start = datetime.now()
iters = 0

for i in range(len(artists)):
    source = artists[i]
    lengths = nx.single_source_shortest_path_length(G, source)
    for j in range(len(artists)):
        target = artists[j]
        if target in lengths:
            distance[i][j] = lengths[target]
        else:
            distance[i][j] = np.inf
    iters += 1
    
print(datetime.now() - start)

0:10:22.009404


In [28]:
import sklearn.cluster

similarity = np.exp(-distance / distance[np.isfinite(distance)].std())
clustering = sklearn.cluster.SpectralClustering(n_clusters=8, affinity='precomputed').fit_predict(similarity)

artist_cluster = {}
cluster_artists = {}
for i in range(len(artists)):
    artist_cluster[artists[i]] = clustering[i]
    if clustering[i] not in cluster_artists:
        cluster_artists[clustering[i]] = [artists[i]]
    else:
        cluster_artists[clustering[i]].append(artists[i])

c:\python36\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [35]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in cluster_artists:
    genre_count = dict()
    total_tracks = 0
    for artist in cluster_artists[cluster]:
        for genre in artist_genres[artist]:
            if genre in genre_count:
                genre_count[genre] += len(artist_tracks[artist])
            else:
                genre_count[genre] = len(artist_tracks[artist])
        total_tracks += len(artist_tracks[artist])
    for genre, count in genre_count.items():
        counts['cluster'].append(cluster)
        counts['genre'].append(genre)
        counts['count'].append(count / total_tracks)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = dict()
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = []
    cluster = playlists[name]
    for artist in cluster_artists[cluster]:
        playlist_tracks[name] += artist_tracks[artist]

In [38]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b69a9985fa8842deb0691b2d0e3f0b69&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2F&scope=playlist-modify-private in your browser




Enter the URL you were redirected to:  http://localhost/?code=AQAuHAsoc-0l-GAQRcs2khImY8Q7DCCQSlIx2jszu5Z4QNC5ZGaQmfghLczReteAouZNEj8sK0Xs2Idv9ZE9Y0-LjjmAbqJA3Hsp7XkOrODKXrWPlD4uhuu3Q4dxvp_ijGDErNZEfys0tmE_VKgO_FkXHh4P-Qh-CVSLweUPC_GQ5r0vpicOT-Iqjuc9m9XAtB8Sgj25cRC0YmcnDwesQdjP


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)